In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# import os
# os.environ['WANDB_DISABLED'] = 'true'

In [3]:
# !pip install evaluate

In [4]:
# !pip install git+https://github.com/huggingface/transformers.git

In [5]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("tokenizer-mz/mz400-tokenizer-bert/mz400-tokenizer-bert", max_len=512)

/home/csenitmz/anaconda3/envs/py310/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from transformers import AutoTokenizer, BertForMaskedLM, BertConfig

model = BertForMaskedLM(config=BertConfig())

In [7]:
# model

In [8]:
%%time
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="datasets/v1-v2-combine_400mb.txt",
    block_size=512,
)

/home/csenitmz/anaconda3/envs/py310/lib/python3.10/site-packages/transformers/data/datasets/language_modeling.py:119: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


CPU times: user 22min 1s, sys: 1min 11s, total: 23min 12s
Wall time: 2min 3s


In [9]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [10]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="models/test-mlm",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_gpu_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [11]:
model.num_parameters()

109514298

In [ ]:
%%time
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
/home/csenitmz/anaconda3/envs/py310/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/csenitmz/anaconda3/envs/py310/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, b

Step,Training Loss
500,7.216300
1000,6.801300
1500,6.770400
2000,6.750300
2500,6.746000
3000,6.738600
3500,6.754500
4000,6.720600
4500,6.723300
5000,6.740600


/home/csenitmz/anaconda3/envs/py310/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/csenitmz/anaconda3/envs/py310/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/csenitmz/anaconda3/envs/py310/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/home/csenitmz/anaconda3/envs/py310/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: Us

In [ ]:
trainer.save_model("models/model-mlm-280124")

In [ ]:
print("completed")

In [ ]:
# !python /kaggle/input/transformers-4-37-1/transformers-main/examples/pytorch/language-modeling/run_mlm.py \
#     --model_name_or_path bert-base-uncased \
#     --tokenizer_name ./kaggle/input/my-tokenizer/mz400-tokenizer-bert/mz400-tokenizer-bert \
#     --dataset_name ./kaggle/input/v1-v2-combine-400mb/v1-v2-combine_400mb.txt \
#     --per_device_train_batch_size 8 \
#     --per_device_eval_batch_size 8 \
#     --do_train \
#     --do_eval \
#     --output_dir /kaggle/working/tmp/test-mlm